# Demonstration on Convolutional Neural Networks

<p>Convolutional Neural Networks (CNN) are special, biologically inspired variants of Multi-Layer Perceptrons. They have proven to be effective in areas such as image recognition and classification. There are four main operations that are the basic building blocks of every Convolutional Neural Network:</p>
<ol>
<li>Convolution</li>
<li>Non-Linearity</li>
<li>Pooling or Down-sampling</li>
<li>Classification or Fully Connected Layer</li>
</ol>
<p>In this demonstration we will walk you through creating a CNN with one convolution layer to recognise handwritten digits from the classic MNIST dataset. The architecture of the network that we will create is similar to the figure below (This figure will not be displayed when you export to .pdf).: </p>

<img src="https://i.imgur.com/rgVIeVy.jpg" alt="Demo Architecture" title="Figure 1" width="500" />

<p>We will begin by first importing the necessary python libraries:</p>

In [ ]:
import six.moves.cPickle as pickle
import gzip
import os
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
import random as rd
from theano.tensor.signal import pool
from theano.tensor.nnet import conv2d
%matplotlib inline
print('***** Import complete *****')

<p>Now we shall proceed to load the datasets into tensors. For this demo we will be using the MNIST dataset. Using the entire MNIST dataset would be computationally intensive and may take too much time. Hence, we shall draw a random sample from it with nearly equal representation of all the 10 digits for the training, validation and test sets. For the training and validation set, we take one-third the number of training set samples. Every image can be represented as a matrix of digits where each entry represents a pixel. In the MNIST dataset, the pixels are in grey scale and normalised. The images are of dimension 28 x 28 pixels and are flattened to an array of 784 elements. We load the dataset into the test, validation and training set tensors. If the corresponding pickle (.pkl) file is not present in the current directory, it will be downloaded for you. The dataset is partitioned into:</p>
<ul>
<li>50,000 training samples from which we draw 6000 samples.</li>
<li>10,000 validation samples from which we draw 2000 samples.</li>
<li>10,000 testing samples from which we draw 2000 samples.</li>
</ul>

In [ ]:
dataset = 'mnist.pkl.gz'
data_dir, data_file = os.path.split(dataset)

# Check if data file present
if data_dir == "" and not os.path.isfile(dataset):
    new_path = os.path.join('', dataset)
    if os.path.isfile(new_path) or data_file == 'mnist.pkl.gz':
        dataset = new_path

# Download the file from MILA if not present 
if (not os.path.isfile(dataset)) and data_file == 'mnist.pkl.gz':
    from six.moves import urllib
    origin = (
        'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
    )
    print('Downloading data from %s' % origin)
    urllib.request.urlretrieve(origin, dataset)

print('***** Loading data *****')

# Open the file
with gzip.open(dataset, 'rb') as f:
    try:
        train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
    except:
        train_set, valid_set, test_set = pickle.load(f)

# Sub-sample and store in a tensor
def shared_dataset(data_xy, sample_size=6000, borrow=True):
    data_x, data_y = data_xy
    indices = 0
    if (sample_size < 0):
        print('Sample size too small!')
        return
    try:
        indices = rd.sample(range(0, data_y.shape[0]), sample_size)
    except ValueError:
        print('Sample size exceeds data size.')
    data_x = data_x[indices, :]
    data_y = data_y[indices]

    shared_x = theano.shared(np.asarray(data_x,
                                        dtype=theano.config.floatX),
                             borrow=borrow)
    shared_y = theano.shared(np.asarray(data_y,
                                        dtype=theano.config.floatX),
                             borrow=borrow)
    return shared_x, T.cast(shared_y, 'int32'), (data_x, data_y)

# Default training sample size = 6000 samples
# Max size = 50000
train_size = 6000
test_set_x, test_set_y, test_set = shared_dataset(
                                                  test_set,
                                                  sample_size=train_size//3
                                                 )
valid_set_x, valid_set_y, valid_set = shared_dataset(
                                                     valid_set,
                                                     sample_size=train_size//3
                                                    )
train_set_x, train_set_y, train_set = shared_dataset(
                                                     train_set,
                                                     sample_size=train_size
                                                     )
# Training set dimension: 6000 x 784
print('Training set: %d samples'
      %(train_set_x.get_value(borrow=True).shape[0])) 
# Test set dimension: 2000 x 784
print('Test set: %d samples'
      %(test_set_x.get_value(borrow=True).shape[0]))
# Validation set dimension: 2000 x 784
print('Validation set: %d samples'
      %(valid_set_x.get_value(borrow=True).shape[0]))
print('The training set looks like this: ')
print(train_set[0])
print('The labels looks like this:')
print(train_set[1])

<p>Now, we shall visualise a random training sample. When plotting, we scale the intensity to the range from 0 to 255. The image is of 28 x 28 pixels.</p>

In [ ]:
# Pick a random image
image_plot = rd.randint(0, len(train_set[1]))
# Scaling image to [0, 255]
pixels = np.array(train_set[0][image_plot] * 255, dtype='uint8')
pixels = pixels.reshape((28, 28))
label = train_set[1][image_plot]
plt.title('Label: {label}'.format(label=label))
plt.imshow(pixels, cmap='gray', interpolation='none')
plt.show()

<p>Next we shall create the convolution layer as in the figure above. The primary purpose of the convolution layer is to extract features from the image. Convolution preserves the spatial relationship between pixels by learning image features using small squares of input data (in this case 5 x 5 pixels of data). In the convolution layer, each of the filters are slid over the image by 1 pixel (stride = 1). For every position, we compute element-wise multiplication between the filter and the sub-image under it and add the multiplication outputs to get the final integer which forms a single element of the output matrix. A non-linearity is introduced after every convolution operation. We then perform pooling which is discussed later. This non-linearity helps our convolutional neural network to model data which is non-linear in nature. In this demonstration, we use the non-linear, tanh function. To summarise, our convolutional layer will have the following specifications: </p>
<ul>
<li>9 filters or channels </li>
<li>Each filter will have a 5 x 5 "field of view"</li>
<li>The stride will be of 1.</li>
<li>It will perform 2 x 2 Max Pooling. We will define the pooling function later.</li>
<li>It will use a Tanh non-linearity</li>
<li>The function shall return the outputs from the non-linearised outputs as well as the parameters (weights and biases).
</ul>

In [ ]:
def convLayer(rng, data_input, filter_spec, image_spec, pool_size, activation):
    # Function that defines the convolution layer. Calls the 
    # activation function and then Pooling function.
    
    # Inputs:
    # rng - random number generator used to initialize weights.
    # data_input - symbolic input image tensor.
    # filter_spec - dimensions of filter in convolution layer.
    #               tuple format:(# of channels, depth, height, width)
    # image_spec - specifications of input images.
    #              tuple format:(batch size, color channels, height, width)
    # pool_size - specification of downsampling (pooling) factor.
    #             tuple format: (# of rows, # of columns)
    # activation - activation function to be used.
    
    # Outputs:
    # output - tensor containing activations fed into next layer.
    # params - list containing layer parameters
    
    # Creating a shared variable for weights that are initialised with samples
    # drawn from a gaussian distribution with 0 mean and standard deviation of 
    # 0.1. This is just a random initialisation.
    W = theano.shared(
        np.asarray(rng.normal(loc=0, scale=0.1, size=filter_spec)),
        borrow=True)
    
    # Bias is a 1 D tensor -- one bias per output feature map. 
    # Initialised with zeros.
    b = theano.shared(np.zeros((filter_spec[0],)), borrow=True)
    
    # Convolve input with specifications. This is Theano's convolution
    # function. It takes as input the data tensor, filter weights, filter 
    # specifications and the image specifications. In our example, the
    # dimensions of the output of this operation would be:
    # mini_batch_size x 9 x 24 x 24
    conv_op_out = conv2d(
        input=data_input,
        filters=W,
        filter_shape=filter_spec,
        input_shape=image_spec)
    
    # Add the bias term and use the specified activation function/ 
    # non-linearity.
    # b.dimshuffle returns a view of the bias tensor with permuted dimensions.
    # In this case our bias tensor is originally of the dimension 9 x 1. The
    # dimshuffle operation used below, broadcasts this into a tensor of
    # 1 x 9 x 1 x 1. Note that there is one bias per output feature map.
    layer_activation = activation(conv_op_out + b.dimshuffle('x', 0, 'x', 'x'))
    
    # Perform pooling on the activations. It is required to reduce the spatial
    # size of the representation to reduce the number of parameters and
    # computation in the network. Hence, it helps to control overfitting
    # Output dimensions: (# channels, image height-filter height+1, 
    #                     image width - filter width+1)
    # In our demo, the dimensions would be of mini_batch_size x 9 x 12 x 12
    output = pooling(input_pool=layer_activation, size=pool_size)
    
    # Combine the weights and biases into a single list
    params = [W, b]
    return output, params

<p>Pooling or down-sampling reduces the dimensionality of each of the feature maps (outputs of the filters) while retaining the most important information. We will perform 2 x 2 Max Pooling. This involves taking the largest element in a 2 x 2 window from the feature maps. Pooling is useful as it helps to reduce the computation load by reducing the number of parameters that need to be optimised/learnt while making the network invariant to small transformations and distortions. The pooling function will be called by the <b>convLayer</b> function.<p>

In [ ]:
def pooling(input_pool, size):
    # Function to perform Max-Pooling on each feature map
    
    # Inputs:
    # input_pool - feature maps obtained as output from convolution layer.
    # size - specification of downsampling (pooling) factor.
    #        tuple format: (# of rows, # of columns)
    
    # Outputs:
    # pool_out - pooled output.
    #            dimensions: (# of channels, conv_output_height/#rows,
    #                         conv_output_width/#rows)
    
    pool_out = pool.pool_2d(input=input_pool, ws=size, ignore_border=True)
    return pool_out

<p>Now we shall construct the fully-connected layer. This is modelled on a traditional perceptron and we shall perform the softmax operation for the activations. Every neuron in this layer is connected to every output from the pooling operation. The purpose of this layer is to use the identified features for classifying the input image into various classes (in our case labels from 0-9) based on the training dataset. After the single fully-connected layer, the final predictions are made using the softmax function. This is where the final classification takes place. The sum of the output probabilities is 1. This is ensured by using Softmax as the activation function. The Softmax function takes a vector of arbitrary real values and reduces it to a vector of numbers from 0 to 1 that sum to 1. This function shall return the predicted probabilities for each class, the final prediction (prediction of class whose probability is maximum) and the parameters (W, b)</p>

In [ ]:
def fullyConnectedLayer(data_input, num_in, num_out):
    # Function to create the fully-connected layer and makes use of the
    # output from the previous layer. It is the final layer in the
    # convolutional neural network architecture and comprises of the
    # Softmax activations.
    
    # Inputs:
    # data_input - input for the softmax layer.
    #              Symbolic tensor of dimensions:
    #              (mini_batch_size, # channels * 12 * 12) 
    # num_in - number of input units. Dimensions would be:
    #           (# channels * 12 * 12)
    # num_out - number of output units or number of output labels.
    
    # Outputs:
    # p_y_given_x - class-membership probabilities.
    # y_pred - class with maximal probability
    # params - parameters of the layer
    
    # Creating a shared variable for weights that are initialised with samples
    # drawn from a gaussian distribution with 0 mean and standard deviation of 
    # 0.1. This is just a random initialisation.
    W = theano.shared(
        value=np.asarray(
            rng.normal(loc=0, scale=0.1, size=(num_in, num_out))),
        name='W',
        borrow=True)
                   
    # Creating a shared variable for biases that are initialised with
    # zeros.
    b = theano.shared(
        value=np.zeros((num_out,)),
        name='b',
        borrow=True)
    
    # Compute class-membership probabilities using the Softmax activation
    # function.
    p_y_given_x = T.nnet.softmax(T.dot(data_input, W) + b)
    
    # Class prediction. Find class whose probability is maximal.
    y_pred = T.argmax(p_y_given_x, axis=1)
                   
    # Combine weights and biases into a single list.
    params = [W, b]
    return p_y_given_x, y_pred, params

<p>Now that we have the basic building blocks in place, we focus on learning all the weights in the network. This is done with the help of back-propogation. Our objective is to learn the parameters in such a way that we minimise the total cost. The parameters are updated using Stochastic Gradient Descent so that the error with the new parameters will be lower. The cost that we will minimise is the negative conditional log likelihood, given by: </p> $$ \frac{1}{|\mathcal{D}|} \mathcal{L} (\theta=\{W,b\}, \mathcal{D}) =
            \frac{1}{|\mathcal{D}|} \sum_{i=0}^{|\mathcal{D}|}
                \log(P(Y=y^{(i)}|x^{(i)}, W,b)) $$
           $$ \ell (\theta=\{W,b\}, \mathcal{D}) = -\mathcal{L}(\theta=\{W, b\}, \mathcal{D})$$
<p>Our objective is to learn the parameter values that minimise this cost. Parameters like number of filters, filter sizes, architecture of the network etc; have been fixed beforehand and will not change during the training process.</p>

In [ ]:
def negative_log_lik(y, p_y_given_x):
    # Function to compute the cost that is to be minimised. 
    # Here, we compute the negative log-likelihood.
    
    # Inputs:
    # y - expected class label
    # p_y_given_x - class-membership probabilities
    
    # Outputs:
    # cost_log - the computed negative log-lik cost
    
    # Generate the relevant row indices
    rows = T.arange(y.shape[0])
    
    # Generate the relevant column indices
    cols = y;
    
    # Computing the log probabilities
    log_prob = T.log(p_y_given_x)
    
    # Obtain the mean of the relevant entries. Loss is formally
    # defined over the sum of the individual error terms as in
    # the eqquation above. However, we use mean instead to speed
    # up convergence.
    cost_log = -T.mean(log_prob[rows, cols])
    return cost_log

<p>We also need a function to compute the number of errors (i.e. wrongly classified instances). We shall return the fraction of wrongly classified instances.</p>

In [ ]:
def errors(y, y_pred):
    # Function to compute the fraction of wrongly classified
    # instances.
    
    # Inputs:
    # y - expected class label
    # y_pred - predicted class label
    
    # Outputs:
    # count_error - number of wrongly classified instances
    
    # Counting the number of number of wrong predictions. T.neq
    # function returns 1 if the variables compared are not equal.
    # The mean would return the fraction of mismatches.
    count_error = T.mean(T.neq(y_pred, y))
    return count_error

<p>We can begin to put it all together by first defining some hyper-parameters. The learning rate is a part of the Stochastic Descent algorithm and dictates how much an update would effect the value of the paramaters. If too small learning rates are passed, the algorithm will take a long time to converge on the optimal values. If the learning rate is too large, the algorithm may keep missing the optimal values. The number of epochs is the number of times the entire training data will be iterated over. Also, the mini-batch size is the number of data samples in a mini-batch. We split the training data into smaller 'batches'. These are known as mini-batches and in each iteration of training, we use one mini-batch. There will be more details on mini-batches later in this course. For now, it suffices to know that mini-batches are smaller samples of the training data such that each iteration uses one mini-batch.</p>

In [ ]:
# set learning rate used for Stochastic Gradient Descent
learning_rate = 0.1
# set number of training epochs
num_epochs = 10
# set number of kernels for each convolution layer
# for e.g. 2 layers - [20, 50]. layer1 = 20, layer2 = 50
num_filters = [9]
# set mini-batch size to be used
mini_batch_size = 50

<p>For better performance, we the data into mini-batches. For this example, we use a mini-batch size of 50 samples.</p>

In [ ]:
# Seeding the random number generator
rng = np.random.RandomState(23455)

# Computing number of mini-batches
n_train_batches = train_set_x.get_value(borrow=True).shape[0]
n_valid_batches = valid_set_x.get_value(borrow=True).shape[0]
n_test_batches = test_set_x.get_value(borrow=True).shape[0]
n_train_batches //= mini_batch_size
n_valid_batches //= mini_batch_size
n_test_batches //= mini_batch_size

print('train: %d batches, test: %d batches, validation: %d batches'
      % (n_train_batches, n_test_batches, n_valid_batches))

<p>We can now create a Theano Computation Graph. The following steps create a computation graph for training, validation and testing. In the convolution layer, filtering reduces the image size to (28-5+1) x (28-5+1) = 24 x 24. Maxpooling reduces this further to (24/2) x (24/2) = 12 x 12. Hence, the 4D output tensor from the convolution layer is of shape (mini_batch_size, num_filters[0], 12, 12). This is fed into hidden layer which produces a matrix of (mini_batch_size, num_filters[0] \* 12 \* 12) = (mini_batch_size, 1296). This is then fed into the Softmax layer which produces the probabilities of the 10 classes for each training sample in the batch.</p>

In [ ]:
# mini-batch index
mb_index = T.lscalar()
# rasterised images
x = T.matrix('x')
# image labels
y = T.ivector('y')

print('***** Constructing model ***** ')

# Reshaping matrix of mini_batch_size set of images into a 
# 4-D tensor of dimensions: mini_batch_size x 1 x 28 x 28
layer0_input = x.reshape((mini_batch_size, 1, 28, 28))

# First convolution and pooling layer
# 4D output tensor is of shape:
# mini_batch_size x 9 x 12 x 12
[layer0_output, layer0_params] = convLayer(
    rng,
    data_input=layer0_input,
    image_spec=(mini_batch_size, 1, 28, 28),
    filter_spec=(num_filters[0], 1, 5, 5),
    pool_size=(2, 2),
    activation=T.tanh)

# Flatten the output into dimensions:
# mini_batch_size x 1296
fc_layer_input = layer0_output.flatten(2)

# The fully connected layer operates on a matrix of 
# dimensions: mini_batch_size x 1296
# It clasifies the values using the softmax function.
[p_y_given_x, y_pred, fc_layer_params] = fullyConnectedLayer(
    data_input=fc_layer_input,
    num_in=num_filters[0]*12*12,
    num_out=10)

# Cost that is minimised during stochastic descent. 
cost = negative_log_lik(y=y, p_y_given_x=p_y_given_x)

# Creates a Theano function that computes the mistakes on the validation set.
# This performs validation.

# Note: the givens parameter allows us to separate the description of the
# Theano model from the exact definition of the inputs variable. The 'key'
# that is passed to the graph is subsituted with the data from the givens
# parameter. In this demo we built the model with a regular Theano tensor
# and we use givens to speed up the GPU. We swap the input index with a 
# slice corresponding to the mini-batch of the dataset to use.

# mb_index is the mini_batch_index
valid_model = theano.function(
    [mb_index],
    errors(y, y_pred),
    givens={
        x: valid_set_x[
            mb_index * mini_batch_size:
            (mb_index + 1) * mini_batch_size
            ],
        y: valid_set_y[
            mb_index * mini_batch_size:
            (mb_index + 1) * mini_batch_size
            ]})

# Create a Theano function that computes the mistakes on the test set.
# This evaluated our model's accuracy.
test_model = theano.function(
    [mb_index],
    errors(y, y_pred),
    givens={
        x: test_set_x[
            mb_index * mini_batch_size:
            (mb_index + 1) * mini_batch_size
            ],
        y: test_set_y[
            mb_index * mini_batch_size:
            (mb_index + 1) * mini_batch_size
            ]})

<p>We create a list of all model parameters to be fit by gradient descent and compute the gradients.</p>

In [ ]:
# List of parameters to be fit during training
params = fc_layer_params + layer0_params
# Creates a list of gradients
grads = T.grad(cost, params)

<p>Below, train_model is a function that updates the model parameters by Stochastic Gradient Descent. We create the updates list by looping over all ( params[i], grads[i] ) pairs. </p>

In [ ]:
# Creates a function that updates the model parameters by SGD.
# The updates list is created by looping over all 
# (params[i], grads[i]) pairs.
updates = [(param_i, param_i - learning_rate * grad_i)
           for param_i, grad_i in zip(params, grads)]

# Create a Theano function to train our convolutional neural network.
train_model = theano.function(
    [mb_index],
    cost,
    updates=updates,
    givens={
        x: train_set_x[
            mb_index * mini_batch_size:
            (mb_index + 1) * mini_batch_size
            ],
        y: train_set_y[
            mb_index * mini_batch_size:
            (mb_index + 1) * mini_batch_size
            ]})

<p>Before we can begin training our model, we need to define some helper functions to help with plotting the cost after each iteration as well as the accuracy and state of the filters after each epoch. We shall plot the filter weights. The weights are useful to visualize because well-trained networks usually display nice and smooth filters without any noisy patterns. Noisy patterns can be an indicator of a network that has not been trained for long enough, or possibly a too low regularization strength that may have led to overfitting. We display the plots during the model training phase and you can observe as they change with each iteration/epoch.</p>

In [ ]:
# Some code to help with the plotting. 
# You don't need to go through the plotting 
# code in detail.

#### Code for the Visualisations #### 
%matplotlib notebook
def update_weight(weights, ax_wt, fig_wt, epoch):
    # global normalise
    weights = np.array(weights[:,0])
    min_channel = np.min(weights)
    max_channel = np.max(weights)
    a = 255/(max_channel - min_channel)
    b = 255 - a * max_channel
    weights = a * weights + b
    for i in range(3):
        for j in range(3):
            idx = 3 * i + j;
            channel = weights[idx]
            ax_wt[i, j].set_xticklabels([])
            ax_wt[i, j].set_yticklabels([])
            ax_wt[i, j].set_xticks([])
            ax_wt[i, j].set_yticks([])
            im = ax_wt[i, j].imshow(channel,
                                    cmap='gray',
                                    interpolation='None') 
    fig_wt.suptitle(
        'Visualisation of Filters After %d Epoch(s)' %epoch, y=1)
    fig_wt.subplots_adjust(right=0.8)
    cbar_ax = fig_wt.add_axes([0.85, 0.15, 0.05, 0.7])
    ch_max = int(np.max(channel))
    ch_min = int(np.min(channel))
    ch_mid = int(ch_min + (ch_max - ch_min)/2)
    cbar = fig_wt.colorbar(im, cax=cbar_ax, ticks=[ch_min+2, ch_mid, ch_max-2])
    cbar.ax.set_yticklabels(["{:.3f}".format(min_channel),
                             "{:.3f}".format(min_channel+
                                             (max_channel-min_channel)/2),
                             "{:.3f}".format(max_channel)])
    fig_wt.canvas.draw()
            
def update_line(line1, fig, x, y):
    line1.set_xdata(np.append(line1.get_xdata(), x))
    line1.set_ydata(np.append(line1.get_ydata(), y))
    fig.canvas.draw()

def update_cost_plot(line2, fig, x, y):
    line2.set_xdata(np.append(line2.get_xdata(), x))
    line2.set_ydata(np.append(line2.get_ydata(), y))
    fig.canvas.draw() 

<p>We can now begin training the model. In this example, we go over the training set 10 times (10 epochs). From the plots we can see that as training progresses, the cost and prediction error decreases. The cost seems to oscillate about a fixed value. Also, we can also notice that the randomness in the filter weights decreases during the training process. This in a way shows our model improving its knowledge on the dataset. <b>Note: Training can take a while. Be patient!</b> </p>

In [ ]:
%matplotlib notebook
iter = 0
epoch = 0
cost_ij = 0
valid_losses = [valid_model(i) for i in range(n_valid_batches)]
valid_score = np.mean(valid_losses)
plt.ion()
fig = plt.figure(figsize=(8, 6))

ax = fig.add_subplot(211)
line1, = ax.plot(epoch, valid_score, 'b-')
ax.set_xlim(0, num_epochs)
ax.set_ylim(0, 1.1)
ax.set_xlabel('Epochs')
ax.set_ylabel('Prediction Error')
ax.set_title('Prediction Error vs. Number of Epochs')

ax2 = fig.add_subplot(212)
line2, = ax2.plot(iter, cost_ij, 'r-')
ax2.set_xlabel('Iterations')
ax2.set_ylabel('Cost')
ax2.set_title('Cost vs. Iterations')
ax2.set_xlim(0, num_epochs * n_train_batches)
ax2.set_ylim(0, 4)
plt.tight_layout()
fig.show()

fig_wt, ax_wt = plt.subplots(3, 3, sharex='col', sharey='row',
                            gridspec_kw=dict(wspace=0.02,
                                             hspace=0.02, 
                                             top=0.95,
                                             bottom=0.05,
                                             left=0.17,
                                             right=0.845))

fig_wt.show()
plots_conv = layer0_params[0].get_value()
plots_fc = fc_layer_params[0].get_value()
update_weight(plots_conv, ax_wt, fig_wt, epoch)
plots_conv = np.array(plots_conv[:, 0])
biases_fc = fc_layer_params[1].get_value()

# Create buffer arrays for the channel and fully connected
# layer weights. These are for visualisation only and are
# chosen arbitrarily!
plot_arr1 = np.array([plots_conv[:, 1, 1]]);
plot_arr2 = np.array([plots_conv[:, 2, 2]]);
plot_arr3 = np.array([plots_conv[:, 3, 3]])
plot_fc1 = np.array([plots_fc[1, 1]])
plot_fc2 = np.array([plots_fc[2, 2]])
plot_fc3 = np.array([plots_fc[3, 3]])

# Create a buffer array for the biases. These are for 
# visualisation only!
plot_biases_arr = np.array([np.transpose(layer0_params[1].get_value())])
plot_biases_fc_arr = np.array([np.transpose(biases_fc[0:5])])

#### End of Visualisations Code ####

# This is where we call the previously defined Theano functions. 
print('***** Training model *****')
while (epoch < num_epochs):
    epoch = epoch + 1
    for minibatch_index in range(n_train_batches):
        # Compute number of iterations performed or total number
        # of mini-batches executed.
        iter = (epoch - 1) * n_train_batches + minibatch_index
        
        # Perform the training of our convolution neural network.
        # Obtain the cost of each minibatch specified using the 
        # minibatch_index.
        cost_ij = train_model(minibatch_index)
        
        # Update the visualisation.
        update_cost_plot(line2, fig, iter, cost_ij)
        
        # Obtain the weights for visualisation
        plots_conv = layer0_params[0].get_value()
        plots_fc_wt = fc_layer_params[0].get_value()
        biases_fc = fc_layer_params[1].get_value()
        plots_wt = np.array(plots_conv[:, 0])
        
        # Append weights to the buffer arrays for visualisation
        plot_arr1 = np.append(plot_arr1, [plots_wt[:, 1, 1]], axis=0)
        plot_arr2 = np.append(plot_arr2, [plots_wt[:, 2, 2]], axis=0)
        plot_arr3 = np.append(plot_arr3, [plots_wt[:, 3, 3]], axis=0)
        plot_fc1 = np.append(plot_fc1, [plots_fc_wt[1, 1]], axis=0)
        plot_fc2 = np.append(plot_fc2, [plots_fc_wt[2, 2]], axis=0)
        plot_fc3 = np.append(plot_fc3, [plots_fc_wt[3, 3]], axis=0)
        plot_biases_arr = np.append(
                               plot_biases_arr,
                               [np.transpose(layer0_params[1].get_value())],
                               axis=0)
        plot_biases_fc_arr = np.append(
                                  plot_biases_fc_arr,
                                  [np.transpose(biases_fc[0:5])],
                                  axis = 0)
    
    # Obtain the weights of the first convolutional layer.
    conv_weights = layer0_params[0].get_value()
    
    # Update the visualisation
    update_weight(conv_weights, ax_wt, fig_wt, epoch)
    
    # Compute the prediction error on each validation mini-batch by
    # calling the previously defined Theano function for validation.
    valid_losses = [valid_model(i) for i in range(n_valid_batches)]
    
    # Compute the mean prediction error across all the mini-batches.
    valid_score = np.mean(valid_losses)
    
    # Update the visualisation.
    update_line(line1, fig, epoch, valid_score)
        
print('***** Training Complete *****')

<p>Let's visualise how the values of three weights in a channel change with iterations. We can observe that each of the weights converge to a value after some iterations. We will also visualise the evolution of the biases of each of the channels.</p>

In [ ]:
%matplotlib inline
plt.ioff()
fig_tr, axarr = plt.subplots(9,figsize=(8, 10), sharex=True)
for ch_count in range(9):
    l1 = axarr[ch_count].plot(plot_arr1[:, ch_count])
    l2 = axarr[ch_count].plot(plot_arr2[:, ch_count])
    l3 = axarr[ch_count].plot(plot_arr3[:, ch_count])
    axarr[ch_count].yaxis.set_label_position("right")
    axarr[ch_count].set_ylabel('Channel '+str(ch_count+1))
    
axarr[0].set_title('Evolution of Three Arbitrary Weights in the 9 Channels')
axarr[8].set_xlabel('Iterations')

fig_tr.subplots_adjust(hspace=0.1)

fig_bias, axarr_bias = plt.subplots(9,figsize=(8, 10), sharex=True)
for ch_count in range(9):
    axarr_bias[ch_count].plot(plot_biases_arr[:, ch_count], 'm-')
    axarr_bias[ch_count].yaxis.set_label_position("right")
    axarr_bias[ch_count].set_ylabel('Channel '+str(ch_count+1))
axarr_bias[0].set_title('Evolution of Biases')
axarr_bias[8].set_xlabel('Iterations')
fig_bias.subplots_adjust(hspace=0.1)
plt.show()

<p>Also, let's visualise some weights and biases from the fully-connected layer.

In [ ]:
fig = plt.figure(figsize=(8, 6))
fig.suptitle('Evolution of Arbitrary Weights and Biases'
             ' in the Fully-Connected Layer')
ax = fig.add_subplot(211)
ax.plot(plot_fc1)
ax.plot(plot_fc2)
ax.plot(plot_fc3)
ax.set_ylabel('Weight Value')

ax2 = fig.add_subplot(212)
for i in range(5):
    ax2.plot(plot_biases_fc_arr[:, i])
ax2.set_ylabel('Bias value')
ax2.set_xlabel('Iterations')
plt.show()

<p>After the training is complete, we can evaluate the prediction accuracy of the trained Convolutional Neural Network over the unseen training set. This will give us a measure of how our trained model will perform on new unseen data.</p>

In [ ]:
# Compute prediction errors on the test set for each mini-batch by
# calling the previous defined Theano function for evaluating our
# model's performance.
test_losses = [test_model(i) for i in range(n_test_batches)]

# Compute the mean prediction error across all the mini-batches.
test_score = np.mean(test_losses)
print('Prediction error: %f %%' % (test_score * 100.))

<p>The operations performed in the convolution layer are analogous to passing the image through several different filters. Different forms of the original image are obtained simply by changing the values in these filters (or more formally kernels). It is important to note that the convolution operation captures the local dependencies in the original image. The CNN learns the values of these filters on its own during the training process. The larger the number of filters we have, the more image features get extracted and the better our network becomes at recognizing patterns in unseen images. To make this clear, we shall take a random image from the test dataset and compare it to the outputs of the convolution layer. We have 9 filters in our convolution layer, hence there will be 9 different images. Notice below how the different filters generate different feature maps from the same original image. Also, note that the size of the image after the convolution layer is smaller as we have performed pooling/down-sampling</p>

In [ ]:
# Code for visualising the activations of the convolution
# layer for a random input number. The objective of this 
# section is to give you an idea of what happens when
# our network reads a new digit to classify. We will 
# visualise the outputs from each filter and the combined
# prediction probabilities. Observe the plots carefully!

# Note: the code below is similar to that discussed above.
# However, a detailed understanding of this section is not
# required. 
%matplotlib inline
plt.ioff()
# Choose a random image
image_plot = rd.randint(0, (train_size//3)-mini_batch_size)
# Create function to compute class-membership
# prediction probabilites.
f2 = theano.function([], p_y_given_x, givens={
        x: test_set_x[image_plot:image_plot+mini_batch_size]})

predictions = f2()
pred_image = predictions[0] * 100

# Parameters for bar chart of probabilities
N = len(pred_image)
x_axis = range(N)
width = 1/1.5

# Obtain convolution layer activations for sample
W_new = layer0_params[0].get_value()
b_new = layer0_params[1].get_value()
x_new = T.matrix('x')
inp_new = x.reshape((1, 1, 28, 28))

# Perform convolution operation
conv_op_out = conv2d(
       input=inp_new,
       filters=W_new,
       filter_shape=(num_filters[0], 1, 5, 5),
       input_shape=(1, 1, 28, 28))

# Pool the output
pooled_out = pooling(input_pool=conv_op_out,
                     size=(2,2))

# Obtain activations
output = T.tanh(pooled_out)

# Create function to obtain activations of conv layer
f = theano.function([], output, givens={
       x: test_set_x[image_plot:image_plot+1]})
test = f()

# Plotting code for sample, prediction probabilties and
# convolution layer activations
fig_actual = plt.figure(figsize=(8, 8))
ax_actual = fig_actual.add_subplot(121)
fig_actual.subplots_adjust(wspace=2)
pixels = np.array(test_set[0][image_plot] * 255,
                     dtype='uint8')
pixels = pixels.reshape((28, 28))
label = test_set[1][image_plot]
ax_actual.set_xticklabels([])
ax_actual.set_yticklabels([])
ax_actual.set_xticks([])
ax_actual.set_yticks([])
ax_actual.set_title(
    'Actual Input image. Label: {label}'.format(label=label))
ax_actual.imshow(pixels,
                 cmap='gray',
                 interpolation='none')

ax_prob = fig_actual.add_subplot(122)
ax_prob.set_yticks(x_axis)
ax_prob.barh(x_axis,
             pred_image,
             width,
             color='blue',
             align='center')
ax_prob.spines['right'].set_visible(False)
ax_prob.spines['top'].set_visible(False)
ax_prob.set_ylabel('Digits')
ax_prob.set_xlabel('Prediction percentage')
ax_prob.set_title('Probabilities of Prediction')
fig_actual.tight_layout()
plt.show()

fig_conv, ax_conv = plt.subplots(3, 3, figsize=(8, 8))
fig_conv.suptitle('Activations of First Convolution Layer')

# Normalisation
activations = np.array(test[0, :])
max_activ = np.max(activations)
min_activ = np.min(activations)
a = 255/(max_activ-min_activ)
b = 255 - a * max_activ
pixels = a * activations + b

for row in range(3):
    for col in range(3):
        idx = row*3 + col
        im = ax_conv[row, col].imshow(pixels[idx],
                                      cmap='gray',
                                      interpolation='none')
        ax_conv[row, col].set_xticklabels([])
        ax_conv[row, col].set_yticklabels([])
        ax_conv[row, col].set_xticks([])
        ax_conv[row, col].set_yticks([])
fig_conv.subplots_adjust(right=0.8)
cbar_conv = fig_conv.add_axes([0.85, 0.15, 0.05, 0.7])

cbar = fig_conv.colorbar(im, cax=cbar_conv, ticks = [100, 150, 220])
cbar.ax.set_yticklabels(["{:.3f}".format(min_activ),
                         "{:.3f}".format(min_activ+
                                        (max_activ-min_activ)/2),
                         "{:.3f}".format(max_activ-0.1)])
plt.show()

<p>From the above experiment, we can see that our simple Convolutional Neural Network is able to achieve reasonable accuracy with a single Convolution layer. In the home assignment, you will create a deep Convolutional Neural Network and explore various hyper parameter settings to improve the accuracy. Good luck!</p>